In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt

import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread, imshow
import cv2

%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from glob import glob
import pandas as pd
from subprocess import check_output
import os
# GOOGLE COLAB: device = torch.device('cuda') 

Directly load the train .jpg images:

In [22]:
basepath = 'sample_train_cervix_shapes/'
transform = transforms.Compose( [transforms.RandomResizedCrop(256),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# change from CIFAR: adding .ImageFolder (allows us to load images directly and allows us
    # to transform images!)
trainset_1 = datasets.ImageFolder(root=basepath,
                                           transform=transform)
    

# IMPORTANT: train_loader needs to take a TENSOR as input
train_loader_1 = torch.utils.data.DataLoader(trainset_1, batch_size=4,
                                          shuffle=True, num_workers=2)
print(trainset_1)
print(type(train_loader_1))
### iterating through a loader to see what its data looks like: ###
# for i, (data, labels) in enumerate(train_loader):
#     print("i",i) --> starts at 0,  <class 'int'>
#     print("data ", data) --> feature tensor,  <class 'int'>
#     print("labels: ",labels) --> label tensor, <class 'list'>

Dataset ImageFolder
    Number of datapoints: 60
    Root location: sample_train_cervix_shapes/
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
<class 'torch.utils.data.dataloader.DataLoader'>


Load tensors instead of .jpg images in order to be able to preprocess data

In [23]:
### 1. Create dataframe where you have each image path, file type and label: ###
all_cervix_images = []

for path in sorted(glob(basepath + "*")): # iterate through each directory (ie each type)
    cervix_type = path.split("/")[-1] # gives you the type (ie directory) of current instance
    cervix_images = sorted(glob(basepath + cervix_type + "/*")) # gives you all images in current directory,
        # in the following format: sample_train_cervix_shapes/Type1/image_number.jpg
    all_cervix_images = all_cervix_images + cervix_images # list of all of the images in same format as previous!
    
all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)

### 2. Convert the 'type' column info to floats so that it can then be put into tensors! ###
all_cervix_images.type[all_cervix_images.type == 'Type1'] = 1
all_cervix_images.type[all_cervix_images.type == 'Type2'] = 2
all_cervix_images.type[all_cervix_images.type == 'Type3'] = 3
all_cervix_images.head()

,imagepath,filetype,type
0,sample_train_cervix_shapes/Type1/191.jpg,jpg,1
1,sample_train_cervix_shapes/Type1/201.jpg,jpg,1
2,sample_train_cervix_shapes/Type1/205.jpg,jpg,1
3,sample_train_cervix_shapes/Type1/208.jpg,jpg,1
4,sample_train_cervix_shapes/Type1/215.jpg,jpg,1


In [42]:
### 3. create feature tensor for each image ###

def get_image_data(image_path):
    """
    Method to get image data as np.array specifying image id and type
    """
    #fname = get_filename(image_id, image_type)
        # fname = sample_train_cervix_shapes/Type{1, 2, 3}/???.jpg
    img = cv2.imread(image_path)
        # cv2.imread(sample_train_cervix_shapes/Type{1, 2, 3}/???.jpg)
    #assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def transform_image(img, rescaled_dim_x, rescaled_dim_y, to_gray=False, normalized=False):
    resized = cv2.resize(img, (rescaled_dim_x, rescaled_dim_y), cv2.INTER_LINEAR)

    if to_gray:
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY).astype('float')
    else:
        resized = resized.astype('float')
        
    if normalized:
        resized = cv2.normalize(resized, None, 0.0, 1.0, cv2.NORM_MINMAX)
        timg = resized.reshape(1, np.prod(resized.shape))
        # return timg/np.linalg.norm(timg)
    return resized

complete_images = []
import glob
import os.path
from os import path
for file in list(glob.glob('sample_train_cervix_shapes/Type*/*')):
    if file.find("Type1")!= -1:
        label = 1
    if file.find("Type2")!= -1:
        label = 2
    if file.find("Type3")!= -1:
        label = 3
    complete_image = get_image_data(file)
    complete_image = transform_image(complete_image, 256, 256, normalized=True)
    complete_image = np.append(complete_image, label)
    #complete_image = tf.convert_to_tensor(complete_image)
    complete_images.append(complete_image)

In [43]:
#type(complete_images)
#type(complete_images)
#complete_images = np.asarray(complete_images)
#type(all_cervix_images['type'])
#import tensorflow as tf
#complete_images = tf.convert_to_tensor(complete_images)
#type(complete_images)
#torch.tensor(complete_images)
#labels_tensor = torch.tensor(all_cervix_images['type'])
#type(labels_tensor)
#trainset_2 = torch.cat([complete_images,labels_tensor],1)

#labels = all_cervix_images['type']
#print(type(complete_images))
#print(type(labels))
train_loader = torch.utils.data.DataLoader(complete_images, batch_size=4,shuffle=True, num_workers=2)

In [44]:
train_loader.dataset
#labels_tensor = torch.tensor(all_cervix_images['type'])
#trainset = torch.cat([images_tensor,labels_tensor],1)
#train_loader1 = torch.utils.data.DataLoader(trainset2, batch_size=4,
 #                                         shuffle=True, num_workers=2)


#for i, data in enumerate(train_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    #print(data)
#     inputs, labels = data
   # print(inputs)
#labels = all_cervix_images['type']
#print(labels)

[array([0.17670683, 0.15261044, 0.19277108, ..., 0.37751004, 0.37751004,
        1.        ]),
 array([0.47058824, 0.45098039, 0.60784314, ..., 0.38039216, 0.35686275,
        1.        ]),
 array([0.19211823, 0.13300493, 0.22167488, ..., 0.27586207, 0.44334975,
        1.        ]),
 array([0.59349593, 0.32520325, 0.3902439 , ..., 0.29268293, 0.38617886,
        1.        ]),
 array([0.02352941, 0.00784314, 0.00392157, ..., 0.        , 0.01568627,
        1.        ]),
 array([1.        , 1.        , 1.        , ..., 0.15853659, 0.19512195,
        1.        ]),
 array([0.        , 0.        , 0.        , ..., 0.01568627, 0.01176471,
        1.        ]),
 array([0.36507937, 0.28571429, 0.27777778, ..., 0.03968254, 0.07936508,
        1.        ]),
 array([0.02352941, 0.01568627, 0.02745098, ..., 0.00392157, 0.02352941,
        1.        ]),
 array([0.01960784, 0.01176471, 0.05098039, ..., 0.02352941, 0.05490196,
        1.        ]),
 array([0.        , 0.00392157, 0.01176471, ..., 0

In [65]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
  def __init__(self):

    super(Net, self).__init__()
    #self.conv1 = nn.Conv2d(in_channels=3, out_channels=7, kernel_size=5)
    #self.pool = nn.MaxPool2d(2,2)
    #self.conv2 = nn.Conv2d(in_channels=7, out_channels=16, kernel_size=5)
    self.linear1 = nn.Linear(256 * 256 * 3, 128) # 196608
    self.linear2 = nn.Linear(128, 1)

  def forward(self, x):

    # convolution layers
    #x = self.pool(F.relu(self.conv1(x)))
    #x = self.pool(F.relu(self.conv2(x)))
    
    # linear layer
    #x = x.view(-1, 16 * 5 * 5) # flatten for linear layers
    x = F.relu(self.linear1(x))
    x = F.softmax(self.linear2(x), dim=1)
    
    return x

# create network depending on hardware available
#if use_gpu:
 # net = Net().cuda()

#else:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [66]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    batch_size = 4
    for i, data in enumerate(train_loader, 0):
        labels = []
        inputs = []
        for j in range(batch_size):
            labels.append(data[j,-1])
        for j in range(batch_size):
            inputs.append(data[j,0:-1])
            
        ### safety check: ###
            # print("len data",len(data[0]))
            # print("len inputs",len(inputs[0]))
            # len inputs = len data - 1 
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        '''
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        '''
print('Finished Training')

AttributeError: 'list' object has no attribute 'dim'